# IMPORTING SOME REQUIRED LIBRARY 

In [6]:
import numpy as np
import pandas as pd
import warnings 

In [7]:
warnings.filterwarnings("ignore")

# GETTING DATASET

In [9]:
#1.ACCESING THE u.data FILE FROM THE DATASETS
col_name=["user_id","item_id","rating","timestamp"]#assigning name to columns
df=pd.read_csv("D:\Study\MACHINE LEARNING\project1\ml-100k\\u.data",sep='\t',names=col_name)
# files data are seprated with tab so here we mention the seprater which is in our data
df.head(10)
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
#checking number of unique user_id{i.e user}in dataset
df["user_id"].nunique()

In [ ]:
#checking the number of unique item_id{i.e movies} in dataset
df["item_id"].nunique()

In [ ]:
#2.ACCESSING THE u.item FROM DATASET
movie_titles=pd.read_csv("D:\\Study\\project1\\ml-100k\\u.item",sep="\|",header=None,encoding_errors= 'replace')
movie_titles.head()

In [ ]:
#WE NEED ONLY 1ST AND 2ND COLUMN OF u.item dataset

movie_titles=movie_titles[[0,1]]
movie_titles.columns=["item_id","Title"]
movie_titles.head()

In [ ]:
#merging the 2 tables data we accessed earlier
#we need user_id,item_id,rating,timestamp,title
#we use pandas merge function
df2=pd.merge(df,movie_titles,on="item_id")
df2.head(10)
df2.tail(10)

# DATA ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#calculated the mean of data frame which is group by title
df2.groupby("Title").mean()

In [ ]:
#accesing the rating column from the mean of grouped movie
df2.groupby("Title").mean()["rating"]

In [ ]:
#sorting the dataframe we get above 
#to see highest rated movie
df2.groupby("Title").mean()["rating"].sort_values(ascending=False)

In [ ]:
#counting the number reviews
#to see which movie has highest number reviews
df2.groupby("Title").count()["rating"].sort_values(ascending=False)

In [ ]:
#CREATING RATING DATAFRAME
rating=pd.DataFrame(df2.groupby("Title").mean()["rating"])
rating

In [ ]:
#adding number of rating or count of movie rating in same  rating dataframe
rating["number_of_rating"]=pd.DataFrame(df2.groupby("Title").count()["rating"])
rating

In [ ]:
#sorting out the rating dataframe
rating.sort_values(by="rating",ascending=False)

In [ ]:
#plotoing histogram for number of rating 
plt.figure(figsize=(10,6))
plt.hist(rating["number_of_rating"],bins=70)
plt.xlabel("number_of_rating")
plt.ylabel("how many movies are rated with")
plt.show()

In [ ]:
#plotting histogram for rating 
plt.hist(rating["rating"],bins=70)
plt.xlabel("average rating")
plt.ylabel("no of times")
plt.show()

In [ ]:
#combining two above histograms
sns.jointplot(x="rating",y="number_of_rating",data=rating,alpha=0.5)

In [ ]:
df2

# CREATING MOVIE RECOMMENDATIOIN


In [ ]:
#creating a matrix with rows/index of user id and columns with movie titles
moviemat=df2.pivot_table(index='user_id',columns='Title',values='rating')
moviemat

In [ ]:
#acccesing  user wise rating of the particular movie
starwars_user_rating=moviemat["Star Wars (1977)"]
starwars_user_rating

In [ ]:
#now we have to co fealate this starwars data  to maovie matrix table
similar_to_starwars=moviemat.corrwith(starwars_user_rating)
similar_to_starwars
#the score obtain of the movie is called corelation
# show show these movie is corelated with our given movie 


In [ ]:
#converting given series to dataframe
corr_starwars=pd.DataFrame(similar_to_starwars,columns=["corelations"])
corr_starwars

In [ ]:
#deleting NAN values
corr_starwars.dropna(inplace=True)
corr_starwars

In [ ]:
# arranging  the above data in descending order
corr_starwars.sort_values("corelations",ascending=False)

In [ ]:
#joining corr_starwars dataframe with rating data frame for getting number_of _rating
corr_starwars=corr_starwars.join(rating["number_of_rating"])
corr_starwars

In [ ]:
corr_starwars[corr_starwars['number_of_rating']>100].sort_values("corelations",ascending=False)

# CREATING THE PREDICTION FUNCTION

In [ ]:
def predict_movie(movie_name):
    movie_user_rating=moviemat[movie_name]
    similar_to_movie=moviemat.corrwith(movie_user_rating)
    
    corr_movie=pd.DataFrame(similar_to_movie,columns=["corelations"])
    corr_movie.dropna(inplace=True)
    
    corr_movie=corr_movie.join(rating["number_of_rating"])
    
    predictions=corr_movie[corr_movie['number_of_rating']>100].sort_values("corelations",ascending=False)
    
    return predictions    

In [ ]:
predict=predict_movie("Crash (1996)")
predict.head()